In [1]:
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from tensorflow.keras.layers import Dense , Dropout
from scikeras.wrappers import KerasRegressor 
from tensorflow.keras.callbacks import EarlyStopping
import time
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from keras_tuner import RandomSearch
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import GridSearchCV

In [2]:
#data = pd.read_pickle('C:/Users/erikm/Desktop/Diplomarbeit Erik Marr/Daten/Finish/TPath_300_finish_data.pkl')
data = pd.read_pickle('C:/Users/erikm/Desktop/Diplomarbeit Erik Marr/Daten/Finish_D4_I7000_F9000/TPath_500_finish_data_D4.pkl')

data

,X-Koordinate,Y-Koordinate,Zeitpunkt,Strom,Kraft,Temperatur
0,0.0000,-0.00200,500,7000,9000,669.05
1,0.0000,-0.00192,500,7000,9000,724.42
2,0.0000,-0.00184,500,7000,9000,779.83
3,0.0000,-0.00176,500,7000,9000,835.21
4,0.0000,-0.00168,500,7000,9000,890.44
...,...,...,...,...,...,...
1066,0.0024,0.00168,500,7000,9000,775.40
1067,0.0024,0.00176,500,7000,9000,715.43
1068,0.0024,0.00184,500,7000,9000,645.85
1069,0.0024,0.00192,500,7000,9000,585.87


In [3]:
df = data.drop(data.columns[2:5], axis = 1)
df

,X-Koordinate,Y-Koordinate,Temperatur
0,0.0000,-0.00200,669.05
1,0.0000,-0.00192,724.42
2,0.0000,-0.00184,779.83
3,0.0000,-0.00176,835.21
4,0.0000,-0.00168,890.44
...,...,...,...
1066,0.0024,0.00168,775.40
1067,0.0024,0.00176,715.43
1068,0.0024,0.00184,645.85
1069,0.0024,0.00192,585.87


In [4]:
df1 = df.sample(frac=1, random_state=42)  # Hier wird 42 als Random State verwendet, um die Ergebnisse reproduzierbar zu machen

print(df1)
df_reset = df1.reset_index(drop=True)
df_reset

      X-Koordinate  Y-Koordinate  Temperatur
184        0.00036       0.00048     1441.10
572        0.00132      -0.00112     1225.70
309        0.00072      -0.00176      832.88
930        0.00216      -0.00104     1158.00
711        0.00156       0.00184      642.96
...            ...           ...         ...
330        0.00072      -0.00008     1500.20
466        0.00108      -0.00144     1045.00
121        0.00024      -0.00048     1484.90
1044       0.00240      -0.00008     1259.10
860        0.00192       0.00152      861.75

[1071 rows x 3 columns]


,X-Koordinate,Y-Koordinate,Temperatur
0,0.00036,0.00048,1441.10
1,0.00132,-0.00112,1225.70
2,0.00072,-0.00176,832.88
3,0.00216,-0.00104,1158.00
4,0.00156,0.00184,642.96
...,...,...,...
1066,0.00072,-0.00008,1500.20
1067,0.00108,-0.00144,1045.00
1068,0.00024,-0.00048,1484.90
1069,0.00240,-0.00008,1259.10


In [5]:
label = df_reset["Temperatur"]
# Korrektur: Verwenden Sie den Spaltennamen direkt, ohne Indexierung der columns-Eigenschaft
df1 = df_reset.drop("Temperatur", axis=1)
X = df1
y = label


In [6]:
X

,X-Koordinate,Y-Koordinate
0,0.00036,0.00048
1,0.00132,-0.00112
2,0.00072,-0.00176
3,0.00216,-0.00104
4,0.00156,0.00184
...,...,...
1066,0.00072,-0.00008
1067,0.00108,-0.00144
1068,0.00024,-0.00048
1069,0.00240,-0.00008


In [7]:
y

0       1441.10
1       1225.70
2        832.88
3       1158.00
4        642.96
         ...   
1066    1500.20
1067    1045.00
1068    1484.90
1069    1259.10
1070     861.75
Name: Temperatur, Length: 1071, dtype: float64

In [8]:
 # train_df enthält 80% der Daten, test_df enthält 20% der Daten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)


In [9]:
# Initialisiere einen MinMaxScaler für die Features
scaler_features = MinMaxScaler()
scaler_features2 = MinMaxScaler()
# Skaliere X_train und X_test
X_train_scaled = scaler_features.fit_transform(X_train)
X_test_scaled = scaler_features.transform(X_test)  # Nutze unterschiedliche Skalierungsparameter

# Initialisiere einen SEPARATEN MinMaxScaler für das Ziel, wenn nötig
scaler_target = MinMaxScaler()


# Skaliere y_train und y_test. Beachte, dass y_train.reshape(-1, 1) verwendet wird, da MinMaxScaler 
# erwartet, dass die Eingaben als 2D-Arrays kommen, und Ziele normalerweise als 1D-Arrays vorliegen.
y_train_scaled = scaler_target.fit_transform(y_train.values.reshape(-1, 1))
y_test_scaled = scaler_target.transform(y_test.values.reshape(-1, 1))


In [10]:
X_train_scaled

array([[0.35, 0.88],
       [0.75, 0.88],
       [0.85, 0.02],
       ...,
       [0.2 , 0.86],
       [1.  , 0.52],
       [0.8 , 0.04]])

In [11]:
y_train_scaled.max()

1.0

In [19]:
# Netzwerkarchitektur
model = Sequential([

    Dense(136, activation='relu', input_shape=(2,), kernel_initializer='he_uniform', kernel_regularizer=l2(0.0001)),

    Dense(168, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(0.0001)),
    
    Dense(24, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(0.0001)),
    
    Dense(1 , activation = 'linear')
])

# Optimierer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Modell kompilieren (Verwendung von mean_squared_error als Verlustfunktion für Regression)
model.compile(optimizer=optimizer,
              loss='mean_squared_error',
              metrics=['mae'])  # Metriken für Regression: Mean Absolute Error und Mean Squared Error

# Early Stopping Callback
early_stopping = EarlyStopping(monitor='loss', patience=5, verbose=1, mode='min', restore_best_weights=True)

# Trainingsparameter
batch_size = 75
epochs = 1000

# Modell trainieren (Annahme: X_train, y_train, X_val, y_val sind vordefiniert)
history = model.fit(X_train_scaled, y_train_scaled,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.2,
                    callbacks=[early_stopping])

Epoch 1/1000
10/10 [==============================] - 1s 16ms/step - loss: 0.2899 - mae: 0.3960 - val_loss: 0.2325 - val_mae: 0.3458
Epoch 2/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.2227 - mae: 0.3426 - val_loss: 0.1673 - val_mae: 0.2561
Epoch 3/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.1958 - mae: 0.3127 - val_loss: 0.1627 - val_mae: 0.2591
Epoch 4/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.1803 - mae: 0.2843 - val_loss: 0.1567 - val_mae: 0.2583
Epoch 5/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.1699 - mae: 0.2743 - val_loss: 0.1390 - val_mae: 0.2286
Epoch 6/1000
10/10 [==============================] - 0s 5ms/step - loss: 0.1518 - mae: 0.2560 - val_loss: 0.1337 - val_mae: 0.2331
Epoch 7/1000
10/10 [==============================] - 0s 6ms/step - loss: 0.1412 - mae: 0.2445 - val_loss: 0.1201 - val_mae: 0.2057
Epoch 8/1000
10/10 [==============================] - 0s 5ms/step - loss: 0

In [20]:
results = model.evaluate(X_test_scaled, y_test_scaled, verbose=2)
results

7/7 - 0s - loss: 0.0081 - mae: 0.0075 - 26ms/epoch - 4ms/step


[0.008079618215560913, 0.007530198432505131]

In [21]:
scaled_predicted_values = model.predict(X_test_scaled, verbose = 0)

# Führen Sie die Rücktransformation der skalierten Werte durch
original_predicted_values = scaler_target.inverse_transform(scaled_predicted_values)
original_actual_values = scaler_target.inverse_transform(y_test_scaled)  # y_test sind die skalierten tatsächlichen Werte
print(f' Bsp. Predicted: {original_predicted_values[100]} Actual: {original_actual_values[100]} ')

def calculate_mae(list1, list2):
    # Stelle sicher, dass beide Listen die gleiche Länge haben
    if len(list1) != len(list2):
        raise ValueError("Listen müssen die gleiche Länge haben")

    # Berechne die absolute Differenz zwischen den Elementen der Listen
    differences = [abs(x - y) for x, y in zip(list1, list2)]

    # Berechne den Durchschnitt der absoluten Differenzen
    mae = sum(differences) / len(differences)

    return mae

# Beispiel
list1 = original_predicted_values
list2 = original_actual_values

mae = calculate_mae(list1, list2)
print(f"Durchschnittliche Abweichung (MAE): {mae}")

 Bsp. Predicted: [1154.885] Actual: [1148.8] 
Durchschnittliche Abweichung (MAE): [7.11800051]


In [62]:
mae = history.history['loss']
val_mae = history.history['val_loss']

epochs = range(1, len(mae) + 1)

# MAE Diagramm
plt.figure(figsize=(10, 6))
plt.plot(epochs, mae, 'r', label='Training MSE')
plt.plot(epochs, val_mae, 'b', label='Validation MSE')
plt.title('Training and Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()

plt.show()

# GridSearch

In [64]:
# def build_model(learning_rate=0.001, activation='relu', regularization=0.0001, dropout_rate=0.0):
#     model = Sequential()
#     model.add(Dense(448, activation=activation, input_shape=(2,), kernel_initializer='he_uniform', kernel_regularizer=l2(regularization)))
#     model.add(Dropout(dropout_rate))
# 
#     model.add(Dense(384, activation=activation, kernel_initializer='he_uniform', kernel_regularizer=l2(regularization)))
#     model.add(Dropout(dropout_rate))
# 
#     model.add(Dense(96, activation=activation, kernel_initializer='he_uniform', kernel_regularizer=l2(regularization)))
#     model.add(Dropout(dropout_rate))
# 
#     model.add(Dense(128, activation=activation, kernel_initializer='he_uniform', kernel_regularizer=l2(regularization)))
#     model.add(Dropout(dropout_rate))
# 
#     model.add(Dense(320, activation=activation, kernel_initializer='he_uniform', kernel_regularizer=l2(regularization)))
#     model.add(Dropout(dropout_rate))
# 
#     model.add(Dense(416, activation=activation, kernel_initializer='he_uniform', kernel_regularizer=l2(regularization)))
#     model.add(Dropout(dropout_rate))
#     
#     model.add(Dense(416, activation=activation, kernel_initializer='he_uniform', kernel_regularizer=l2(regularization)))
#     model.add(Dropout(dropout_rate))
#     
#     model.add(Dense(256, activation=activation, kernel_initializer='he_uniform', kernel_regularizer=l2(regularization)))
#     model.add(Dropout(dropout_rate))    
#     
#     model.add(Dense(1, activation='linear'))
#     model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error', metrics=['mae'])
#     return model
# 
# # Verwenden Sie eine Funktion, um das Modell zu instanziieren, für scikit-learn Wrapper
# model = KerasRegressor(model=build_model, verbose=2)
# 
# # Anpassung der Parameter im param_grid
# param_grid = {
#     'model__learning_rate': [0.01, 0.001, 0.0001],
#     'model__regularization': [0.001, 0.0001],
#     'fit__batch_size': [10, 25, 50, 75],
#     'fit__epochs': [50],
#     'model__dropout_rate' : [0.0, 0.1, 0.2]
# }
# 
# grid_search = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, verbose=2)
# # Hinweis: Stellen Sie sicher, dass Ihre Daten (X_train_scaled, y_train_scaled) korrekt definiert sind
# grid_result = grid_search.fit(X_train_scaled, y_train_scaled)
# # Beste Parameter und Score ausgeben
# print("Beste Parameter:", grid_search.best_params_)
# print("Beste Genauigkeit:", grid_search.best_score_)
# 
# with open("Gridsearch_D4.txt", "w") as f:
#     f.write(f"Beste Parameter: {grid_search.best_params_}\n")
#     f.write(f"Beste Genauigkeit: {grid_search.best_score_}")


In [66]:
# Funktion zum Erstellen des Modells
def build_model(hp):
    model = Sequential()
    model.add(Dense(hp.Int('input_units', min_value=8, max_value=328, step=16), input_shape=(2,), activation='relu'))
    for i in range(hp.Int('n_layers', 1, 10)):
        model.add(Dense(hp.Int(f'units_{i}', min_value=8, max_value=328, step=16), activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Durchführung der Random Search dreimal
for run in range(1, 4):
    # Anpassen des Verzeichnisses und des Projektnamens für jeden Durchlauf
    directory = 'random_search'
    project_name = f'random_search_D4_{run}'
    
    tuner = RandomSearch(
        build_model,
        objective='val_loss',
        max_trials=100,
        executions_per_trial=1,
        directory=directory,
        project_name=project_name
    )
    
    # Durchführung des Random Search
    tuner.search(X_train_scaled, y_train_scaled, epochs=50, verbose =0, batch_size=20, validation_split=0.2, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])
    
    # Abrufen und Speichern des besten Modells
    best_model = tuner.get_best_models(num_models=1)[0]
    model_path = os.path.join(directory, project_name, 'best_model.h5') 
    best_model.save(model_path)
    

    # Optional: Abrufen und Ausgeben der besten Hyperparameter
    best_hyperparameters = tuner.get_best_hyperparameters()[0]
    
    # Konvertieren der Hyperparameter in ein DataFrame
    df_hyperparameters = pd.DataFrame([best_hyperparameters.values])
    # Speichern des DataFrame als CSV
    df_hyperparameters.to_csv(f'random_search_D4_{run}.csv', index=False)
    
    print(f"Beste Hyperparameter für Lauf {run}: {best_hyperparameters.values}")


Beste Hyperparameter für Lauf 1: {'input_units': 264, 'n_layers': 7, 'units_0': 232, 'units_1': 280, 'units_2': 264, 'units_3': 216, 'units_4': 72, 'units_5': 216, 'units_6': 88, 'units_7': 40, 'units_8': 120, 'units_9': 248}


C:\Users\erikm\Desktop\Diplomarbeit Erik Marr\Projekt X\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Beste Hyperparameter für Lauf 2: {'input_units': 136, 'n_layers': 9, 'units_0': 72, 'units_1': 200, 'units_2': 296, 'units_3': 248, 'units_4': 72, 'units_5': 328, 'units_6': 296, 'units_7': 264, 'units_8': 8, 'units_9': 328}
Beste Hyperparameter für Lauf 3: {'input_units': 136, 'n_layers': 2, 'units_0': 168, 'units_1': 24, 'units_2': 184, 'units_3': 40, 'units_4': 8, 'units_5': 56, 'units_6': 72}
